# AWD-LSTM

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export 
from exp.nb_12 import *

## Data

In [3]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [4]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])

In [5]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [6]:
proc_tok, proc_num = TokenizeProcessor(max_workers=8), NumericalizeProcessor()

In [7]:
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok, proc_num])

In [8]:
pickle.dump(ll, open(path/'ll_lm.pkl', 'wb'))
pickle.dump(proc_num.vocab, open(path/'vocab_lm.pkl', 'wb'))

In [9]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [10]:
bs, bptt = 64, 70

In [11]:
data = lm_databunchify(ll, bs, bptt)

## AWD-LSTM
![](pictures/lstm.jpg)

To take advantage of the GPU we do one matrix multiplication and split the output into 4 chunks for the 4 gates instead of doing 4 matrix multiplications.

### LSTM from scratch

In [12]:
class LSTMCell(nn.Module):
    def __init__(self, ni, nh):
        super().__init__()
        self.ih = nn.Linear(ni, 4*nh)
        self.hh = nn.Linear(nh, 4*nh)
        
    def forward(self, input, state):
        h, c = state  # (64, 300) each
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)  # (64, 1200) -> 4x * (64, 300) because args of chunk are (chunks, dim)
        
        ingate, forgetgate, outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()
        
        c = (forgetgate * c) + (ingate * cellgate)  # (64, 300)
        h = outgate * c.tanh()                      # (64, 300)
        
        return h, (h,c)

In [13]:
class LSTMLayer(nn.Module):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = LSTMCell(*cell_args)
        
        
    def forward(self, input, state):
        # input (64, 70, 300), state both (64, 300)
        inputs = input.unbind(1)  # (64, 70, 300) -> tuple of len 70 with shape (64, 300)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state  # (64, 70, 300) and 2 tuple of shapes (64, 300)

In [14]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [15]:
x = torch.randn(64, 70, 300)

In [16]:
h = (torch.zeros(64, 300), torch.zeros(64, 300))

CPU:

In [17]:
%timeit -n 10 y, h1 = lstm(x, h)

80.3 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


GPU:

In [18]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [19]:
def time_fn(f):
    f()
    torch.cuda.synchronize()

In [20]:
f = partial(lstm, x, h)
time_fn(f)

In [21]:
%timeit -n 10 time_fn(f)

16.8 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Builtin LSTM version

In [22]:
lstm = nn.LSTM(300, 300, 1, batch_first=True)

In [23]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(1, 64, 300), torch.zeros(1, 64, 300))

CPU:

In [24]:
%timeit -n 10 y, h1 = lstm(x,h)

74.4 ms ± 71.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [26]:
f = partial(lstm, x, h)
time_fn(f)

In [27]:
%timeit -n 10 time_fn(f)

1.76 ms ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


CPU versions almost has the same speed. On the GPU, however, PyTorch uses CuDNN behind the scenes which greatly optimized the for loop.

### Jit version

In [28]:
import torch.jit as jit
from torch import Tensor

In [29]:
class LSTMCell(jit.ScriptModule):
    def __init__(self, ni, nh):
        super().__init__()
        self.ni = ni
        self.nh = nh
        self.w_ih = nn.Parameter(torch.randn(4 * nh, ni))
        self.w_hh = nn.Parameter(torch.randn(4 * nh, nh))
        self.bias_ih = nn.Parameter(torch.randn(4 * nh))
        self.bias_hh = nn.Parameter(torch.randn(4 * nh))

    @jit.script_method
    def forward(self, input:Tensor, state:Tuple[Tensor, Tensor])->Tuple[Tensor, Tuple[Tensor, Tensor]]:
        hx, cx = state
        gates = (input @ self.w_ih.t() + self.bias_ih +
                 hx @ self.w_hh.t() + self.bias_hh)
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, (hy, cy)

In [30]:
class LSTMLayer(jit.ScriptModule):
    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    @jit.script_method
    def forward(self, input:Tensor, state:Tuple[Tensor, Tensor])->Tuple[Tensor, Tuple[Tensor, Tensor]]:
        inputs = input.unbind(1)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs, dim=1), state

In [31]:
lstm = LSTMLayer(LSTMCell, 300, 300)

In [32]:
x = torch.randn(64, 70, 300)
h = (torch.zeros(64, 300),torch.zeros(64, 300))

In [33]:
%timeit -n 10 y,h1 = lstm(x,h)

75.8 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
lstm = lstm.cuda()
x = x.cuda()
h = (h[0].cuda(), h[1].cuda())

In [35]:
f = partial(lstm,x,h)
time_fn(f)

In [36]:
%timeit -n 10 time_fn(f)

5.25 ms ± 50.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


CPU version again has the same speed but with the jit version we almost have the same speed from scratch as CuDNN!

### Dropout
#### RNN Dropout

In [37]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [38]:
x = torch.randn(10, 10)

The weights that are not nullified are corrected by a factor of 1-p:

In [39]:
mask = dropout_mask(x, (10, 10), 0.5); mask

tensor([[0., 2., 2., 0., 0., 2., 2., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2., 0., 0., 2., 2.],
        [2., 0., 2., 0., 2., 2., 0., 0., 2., 0.],
        [0., 2., 0., 0., 2., 2., 0., 0., 0., 2.],
        [0., 0., 2., 2., 2., 0., 0., 2., 0., 2.],
        [2., 2., 0., 2., 0., 0., 0., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0., 2., 0., 0., 2.],
        [0., 2., 2., 2., 2., 2., 2., 0., 2., 2.],
        [0., 0., 0., 2., 2., 0., 2., 0., 0., 0.]])

Applying the dropout mask is simply done by `x * mask`.
Why don't we use PyTorch's dropout? We do not want to nullify all the coefficients randomly: on the sequence dimension we want to always nullify the same positions.

In [40]:
mask.sum()/mask.nelement()

tensor(1.1200)

In [41]:
(x*mask).std(), x.std()

(tensor(1.6349), tensor(1.0754))

Inside the RNN, the tensors have the shape (bs, seq_len, vocab_size). We want to apply the dropout mask across the seq_len dimension, we therefore create a dropout mask for the first and third demension and broadcast it along the seq_len dimension:

In [42]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p
    
    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        
        return x * m

In [43]:
dp = RNNDropout(0.3)
test_input = torch.randn(3, 3, 7)
test_input, dp(test_input).transpose(0,1)  
# transpose to make the seq_len dim come first and visualize that always same fields are nullified

(tensor([[[ 0.5056,  0.0121, -0.1962, -0.6330, -0.2526,  1.0439, -0.6941],
          [ 1.1515,  0.0723,  0.3830,  1.2073, -0.4272, -0.0685, -0.2054],
          [ 1.0184, -0.0118, -0.7056,  1.3648, -0.0542, -0.4178, -1.2435]],
 
         [[ 0.5554, -0.6984,  0.4897,  0.5830, -2.5208, -0.0852,  0.5275],
          [ 0.1852, -0.3760,  2.3418,  0.1264, -0.2487,  2.6627, -0.7511],
          [ 1.2503,  0.9745,  0.0933,  0.4386, -1.1615,  0.6218,  0.1381]],
 
         [[-0.4208,  1.0365,  0.2137,  1.8034,  0.3262,  2.2972,  0.5066],
          [-0.3487,  1.6206,  1.1809, -0.1099, -2.6683,  0.6671, -0.6432],
          [-0.9166,  1.5202, -0.8253,  1.7873,  1.5201, -1.1465, -0.1496]]]),
 tensor([[[ 0.7223,  0.0000, -0.2802, -0.9043, -0.3609,  0.0000, -0.0000],
          [ 0.0000, -0.9977,  0.0000,  0.8328, -0.0000, -0.1217,  0.7535],
          [-0.6011,  0.0000,  0.0000,  2.5762,  0.4660,  3.2818,  0.7237]],
 
         [[ 1.6449,  0.0000,  0.5472,  1.7247, -0.6102, -0.0000, -0.0000],
          [ 0

#### Weight Dropout
Weight Dropout is applied to the weights of the inner LSTM hidden to hidden matrix. Hacky if we want to preserve the CuDNN speed and not reimplement cell from scracth. We add parameter that will contain the raw weights and we replace the weight matrix in the LSTM at the beginnning of each forward pass.

[Use this instead?](https://pytorchnlp.readthedocs.io/en/latest/_modules/torchnlp/nn/weight_drop.html)

In [44]:
#export
import warnings

WEIGHT_HH = 'weight_hh_l0'

class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.], layer_names=[WEIGHT_HH]):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [45]:
module = nn.LSTM(5, 2)  # input_sz, hid_sz

In [46]:
dp_module = WeightDropout(module, 0.4)

**Before applying module:**

In [47]:
getattr(dp_module.module, WEIGHT_HH)

Parameter containing:
tensor([[-0.0058, -0.1290],
        [ 0.0350, -0.3804],
        [ 0.5237,  0.5607],
        [ 0.5398,  0.4805],
        [ 0.1579,  0.6691],
        [-0.1472,  0.0067],
        [ 0.1549, -0.2744],
        [-0.2423,  0.3669]], requires_grad=True)

In [48]:
getattr(dp_module, f'{WEIGHT_HH}_raw')

Parameter containing:
tensor([[-0.0058, -0.1290],
        [ 0.0350, -0.3804],
        [ 0.5237,  0.5607],
        [ 0.5398,  0.4805],
        [ 0.1579,  0.6691],
        [-0.1472,  0.0067],
        [ 0.1549, -0.2744],
        [-0.2423,  0.3669]], requires_grad=True)

**After applying module:**

In [49]:
test_input = torch.randn(4, 20, 5)  # bs, seq_len, hid_sz

In [50]:
h = (torch.zeros(1, 20, 2), torch.zeros(1, 20, 2))

In [51]:
x, h = dp_module(test_input, h)

In [52]:
getattr(dp_module.module, WEIGHT_HH) * (1 - 0.4)

tensor([[-0.0000, -0.1290],
        [ 0.0000, -0.3804],
        [ 0.5237,  0.5607],
        [ 0.0000,  0.0000],
        [ 0.1579,  0.0000],
        [-0.1472,  0.0067],
        [ 0.0000, -0.2744],
        [-0.2423,  0.3669]], grad_fn=<MulBackward0>)

In [53]:
x, h = dp_module(test_input, h)

In [54]:
getattr(dp_module.module, WEIGHT_HH) * (1 - 0.4)

tensor([[-0.0058, -0.1290],
        [ 0.0000, -0.3804],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.4805],
        [ 0.0000,  0.0000],
        [-0.1472,  0.0000],
        [ 0.1549, -0.0000],
        [-0.2423,  0.3669]], grad_fn=<MulBackward0>)

Every time the `dp_module` is called different fields of the weight matrix are nullified.

Original is unchanged

In [55]:
getattr(dp_module, f'{WEIGHT_HH}_raw')

Parameter containing:
tensor([[-0.0058, -0.1290],
        [ 0.0350, -0.3804],
        [ 0.5237,  0.5607],
        [ 0.5398,  0.4805],
        [ 0.1579,  0.6691],
        [-0.1472,  0.0067],
        [ 0.1549, -0.2744],
        [-0.2423,  0.3669]], requires_grad=True)

#### Embedding Dropout
Applies dropout to full rows of the embedding matrix (zeroes embedding for specific words).

In [56]:
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb, self.embed_p = emb, embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1
            
    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0), 1)  # (100, 1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            # list of 100 numbers being eiter 0 or 2
            masked_embed = self.emb.weight * mask  # some words are zeroed
        
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

```
Docstring:
A simple lookup table that looks up embeddings in a fixed dictionary and size.

This module is often used to retrieve word embeddings using indices.
The input to the module is a list of indices, and the embedding matrix,
and the output is the corresponding word embeddings.
```

In [57]:
enc = nn.Embedding(100, 7, padding_idx=1)  # 100 embeddings of size 7

In [58]:
enc_dp = EmbeddingDropout(enc, 0.5)

In [59]:
test_input = torch.randint(0, 100, (3,))

In [60]:
test_input

tensor([80, 74, 77])

In [61]:
enc_dp(test_input)

tensor([[ 1.7836,  1.6946,  3.9820,  2.2164, -3.0690, -3.9632, -2.7137],
        [-2.4584,  2.6614,  3.5521, -2.1865,  2.8898, -1.4482,  3.0444],
        [ 0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000]],
       grad_fn=<EmbeddingBackward>)

Zeroes embedding for specific words (emb_sz = 7).

### AWD-LSTM Main Model
Regular multilayer LSTM with all those kinds of dropout.

In [64]:
#export
def to_detach(h):
    "Detaches h from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [134]:

#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs, self.emb_sz, self.n_hid, self.n_layers = 1, emb_sz, n_hid, n_layers  # bs = 1 to make it reset h
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [nn.LSTM(input_size=emb_sz if l == 0 else n_hid, 
                             hidden_size=(n_hid if l != n_layers - 1 else emb_sz),
                             num_layers=1, batch_first=True
                            ) for l in range(n_layers)]
        """
        (Pdb) self.rnns[0]
        LSTM(300, 500, batch_first=True)
        (Pdb) self.rnns[1]
        LSTM(500, 500, batch_first=True)
        (Pdb) self.rnns[2]
        LSTM(500, 300, batch_first=True)
        (Pdb) self.rnns[3]
        """
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        # Initialize embedding
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs, sl = input.size()
        if bs!=self.bs:  # self.bs = 1 at beginning, forces to reset h
            self.bs=bs
            self.reset()
            
        # self.hidden ist list of len 3 of tuples of len 2 of tensors of shape [1, 64, 500/300]
        # input shape [64, 70]
        raw_output = self.input_dp(self.emb_dp(input))  # [64, 70, 300], dropout on input along seq dimension
        new_hidden,raw_outputs,outputs = [],[],[]

        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            # new_h has shape [1, 64, 500] for l = 0 (final hidden state), raw_output [64, 70, 500] for l == 0
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)  # RNNDropout along seq dim on h's
            outputs.append(raw_output) 
        # here new_hidden is list of len n_layers with tuples of two tensors h,c each
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs  # with and without dropout

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

In [135]:
test_input = torch.randint(low=0, high=30000, size=(64, 70)) # (bs, seq_len)

In [136]:
awd_lstm = AWD_LSTM(30000, 300, 500, 3, vocab.index(PAD))

In [137]:
raw_outputs, outputs = awd_lstm(test_input)

In [138]:
len(raw_outputs), len(outputs)

(3, 3)

In [139]:
raw_outputs[0].shape, outputs[0].shape

(torch.Size([64, 70, 500]), torch.Size([64, 70, 500]))